In [82]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt
# tf.debugging.set_log_device_placement(True)

**！！！<br>下面以“张量”代指泛指的张量，即可能为`Tensor`对象，也可能为`Variable`对象；如不做特殊说明，“变量”特指`Variable`对象**

<center>

# Tensors
</center>

## 

<center>

# Variables
</center>

## 1. `Variable`的创建

一般情况下`tf.Variable(initial_value)`以`initial_value`的数据类型和形状创建`Variable`实例，`initial_value`也可以是不带参数的可调用对象，其在调用时返回用于初始化的对象，这种情况下必须指明构造函数的`dtype`参数；需要注意的是，若调用`init_ops.py`模块中的初始化函数，则该初始化函数必须首先绑定到一个 shape 上；

```python
def init_fc():
    return tf.constant([1, 2, 3])
var = tf.Variable(init_fc)  # Different from `tf.Variable(init_fc())`
assert tf.reduce_all(var == init_fc())

```
对`Variable`构造函数的`shape`参数指明`tf.TensorShape(None)`时，表示该变量的形状待定，进而可以在之后的代码中为其赋值，新赋予的值不必与传递给构造函数`initial_value`形状相同；需要注意的是，无论赋值多少次，`Variable`实例化对象的`shape`属性均为`<unknown>`，这意味着该对象可以多次赋予不同形状的张量；示例如下
```python
var = tf.Variable([[1.], [2.], [3.]], shape=tf.TensorShape(None))
var.assign([[1., 2.], [3., 4.]])
var.assign([1., 2., 3., 4.])
```
从现有`Variable`创建新的`Variable`只会复制其张量值，两个`Variable`不共享内存，即使在创建这些变量时使用了相同的名称，示例如下
```python
var1 = tf.Variable([2.0, 3.0], name="var_unique")
var2 = tf.Variable(var1, name="var_unique")
id(var1) == id(var2)  # ==> False
var1.name, var2.name  # ==> ('var_unique:0', 'var_unique:0')
```
## 2. `Variable`与`Tensor`
`tf.Variable`是由`tf.Tensor`支持的数据结构，其与`Tensor`一样也具有`shape`、`dtype`、`numpy()`等属性，所有为`Tensor`类重载的操作符都适用于`Variable`；但需要说明的是，大多张量操作会返回`Tensor`对象而非`Variable`对象，两者内存地址自然也不相同；而调用`var.assign()`，`var.assign_add()`等函数通常使用原内存地址并返回`Variable`对象；示例如下
```python
var1 = tf.Variable([2.0, 3.0])
orig_id = id(var1)
var1.assign_add([1, 2])
orig_id == id(var1)  # ==> True
var2 = tf.add(var1, [1, 2])
id(var2) == id(var1)  # ==> False
isinstance(var2, tf.Tensor)  # ==> True
```



## 3. Lifecycles, naming, and watching
在基于 Python 的 TF 中，`tf.Variable`实例化对象与其他 Python 对象具有相同的生命周期，当没有对该变量的引用时，它会自动释放；

对变量命名有利于对其跟踪和调试；

保存和加载模型时会对变量名进行保存，默认情况下，模型中的变量将自动获取独有的变量名；

创建变量时指明`trainable=False`可以不对其进行求导，例如训练步长计数器

Variables are often captured and manipulated by `tf.function`s. This works the
same way the un-decorated function would have:
```
>>> v = tf.Variable(0.)
>>> read_and_decrement = tf.function(lambda: v.assign_sub(0.1))
>>> read_and_decrement()
<tf.Tensor: shape=(), dtype=float32, numpy=-0.1>
>>> read_and_decrement()
<tf.Tensor: shape=(), dtype=float32, numpy=-0.2>

Variables created inside a `tf.function` must be owned outside the function
and be created only once:

>>> class M(tf.Module):
...   @tf.function
...   def __call__(self, x):
...     if not hasattr(self, "v"):  # Or set self.v to None in __init__
...       self.v = tf.Variable(x)
...     return self.v * x
>>> m = M()
>>> m(2.)
<tf.Tensor: shape=(), dtype=float32, numpy=4.0>
>>> m(3.)
<tf.Tensor: shape=(), dtype=float32, numpy=6.0>
>>> m.v
<tf.Variable ... shape=() dtype=float32, numpy=2.0>
```

See the `tf.function` documentation for details.
## 4. 部署`Variable`和`Tensor`
为了提高性能，TF 总会在与张量数据类型所兼容的可用的最快设备上部署`Variable`和`Tensor`，即默认`tf.config.set_soft_device_placement(True)`；这意味着在 GPU 可用时，大多数`Variable`会被部署至 GPU 上；尽管如此，用户也可以通过`tf.device()`上下文管理器指定所使用的设备；可以通过设定`tf.debugging.set_log_device_placement(True)`来查看变量部署情况，注意该代码段需要在程序启动时运行
``` python
with tf.device('CPU:0'):
    a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
print(c)
```
尽管手动部署可以运行，但分布式训练可以以一种更方便更灵活的方式来优化计算；
- 更多使用`Variable`的方式参看[自动求导相关指南](#Gradients-and-automatic-differentiation)
- 更多有关分布式训练的方法参见[相关指南](https://www.tensorflow.org/guide/distributed_training)

## 

<center>
    
# Automatic differentiation
</center>

TF 的[自动求导](https://en.wikipedia.org/wiki/Automatic_differentiation)功能主要靠[`tf.GradientTape`](http://localhost:8888/notebooks/Help_Viewer_Python/TensorFlow/TF2/API/tf.x.ipynb#tf.GradientTape())实现；调用`tf.GradientTape`会返回一个上下文管理器对象，它会记录**于其内部**发生在**所追踪的**`Variable`上的每一个操作、操作之间顺序、以及操作在相关张量上的导数值；在反向传播时，“梯度磁带”会逆向追溯这些操作，将相应导数值根据求导法则进行运算，进而实现自动求导；由于相应的导数值在向前传播时便已经计算好，进而反向传播时仅 [!TODO ]



## 1. 计算导数
`tape.gradient(target, sources)`利用“梯度磁带”所记录的操作及相关导数值计算`target`对`sources`的梯度，并返回一个与`sources`嵌套结构完全相同的对象(关于嵌套可参见`tf.nest`)；
- 当`target`为标量时，返回对象中每个元素为`target`对`sources`中的相应元素的导数；

    ```python
    w = tf.Variable(tf.ones((3, 2)), name='w')
    b = tf.Variable(tf.range(1, 3, dtype=tf.float32), name='b')
    x = [[1., 2., 3.]]
    with tf.GradientTape() as tape:
        y = x @ w + b
        loss = tf.reduce_mean(y**2)
    [dl_dw, dl_db] = tape.gradient(loss, [w, {"b": b}])
    isinstance(dl_dw, list) and isinstance(dl_db, dict)  # ==> True
    ```
    这有利于对`tf.Module`对象中变量求导，只需通过调用`Module.trainable_variables`属性将一个模型所有的可训练变量传递给“梯度磁带”，进而能够得到对模型中所有可训练变量的导数；

- 当`target`为多个标量或非标量时，返回对象中每个元素为所有`target`对`sources`中的相应元素的导数的和

    ```python
    x = tf.Variable(2.0)
    with tf.GradientTape(persistent=True) as tape:
        y = x**2
        z = 1 / x
        w = x * [3., 4.]
    assert tape.gradient(y, x).numpy() == 4.0
    assert tape.gradient(z, x).numpy() == -0.25
    assert tape.gradient([y, z], x) == 3.75
    assert tape.gradient(w, x) == 7.0
    del tape   # Drop the reference to the tape
    ```
    这种方法常用于模型经 element-wise 计算得到的损失，且在进行 SGD 时以一个变量的所有梯度值之和为基础作参数更新的情况；如果需要对每个元素求导数值，请参考[雅可比矩阵](https://www.tensorflow.org/guide/advanced_autodiff#jacobians)

这里参数`persistent`用于指明是否创建一个存留的“梯度磁带”；默认情况下“梯度磁带”会在调用`.gradient()`方法后释放所有持有的资源；若需要多次调用`.gradient()`以对多个梯度进行计算，须指明`persistent=True`，此时只有在丢弃相关引用后“梯度磁带”才会释放其内部资源；示例如上；

## 2. 设置`GradientTape`所追踪的对象
`tape.watched_variables()`方法返回所有“梯度磁带”正在追踪的 **`Variable`对象**所组成的元祖；“梯度磁带”默认对所有可训练的`Variable`进行追踪，而不对`Tensor`进行追踪；需要说明的是，尽管`Variable`在进行运算后输出中间结果为`Tensor`实例，但由于“梯度磁带”会记录发生在训练变量上所有操作以及相应梯度值，进而`target`对该中间结果的导数依旧可以求得，例如下面示例中`loss`对`y`的导数；

```python
w = tf.Variable(tf.ones((3, 2)), name='w')
b = tf.Variable(tf.range(1, 3, dtype=tf.float32), name='b')
x = tf.constant([[1., 2., 3.]])
with tf.GradientTape() as tape:
    y = x @ w + b
    assert isinstance(y, tf.Tensor)
    loss = tf.reduce_mean(y**2)
dl_dw, dl_dy, dl_dx = tape.gradient(loss, [w, y, x])
assert dl_dy is not None
assert dl_dx is None
```

可以通过指定`watch_accessed_variables=False`来取消“梯度磁带”对所有变量自动追踪的功能，此时需通过`tape.watch(tensor)`来指定“梯度磁带”追踪哪一个变量；`.watch()`方法也可以作用于`Tensor`对象上，但通过`.watch()`被追踪的`Tensor`并不会被添加在`.watched_variables()`方法所返回的元素中，实例如下
```python
x0 = tf.constant(0.0)
x1 = tf.Variable(10.0)
x2 = tf.Variable(-1.0)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x0)
    tape.watch(x1)
    y0 = tf.math.sin(x0)
    y1 = tf.nn.softplus(x1)
    y = tf.reduce_sum(y0 + y1 + x2)
dy_dx0, dy_dx1, dy_dx2 = tape.gradient(y, [x0, x1, x2])
assert len(tape.watched_variables()) == 1
assert dy_dx2 is None  # x2 is not watched
```
## 3. 控制流
“梯度磁带”的上下文管理器中的 Python 控制流会按其正常执行方式执行，例如下面shiyong`if`语句的例子，只有和操作涉及到的变量才具有梯度值；需要注意的是，控制语句本身是不可导的，进而其对于“梯度磁带”是不可见的，即对于`x`的导数永远是 None；

```python
x = tf.constant(1.0)
v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    if x > 0.0:
        result = v0
    else:
        result = v1**2
g_v0, g_v1, g_x = tape.gradient(result, [v0, v1, x])
assert g_v1 is None and g_x is None
assert g_v0 is not None
```
## 4. 得到梯度值为 None 的情况

当计算图中`target`与`source`是不连通的状态时，对`source`求导会得到 None；除很明显的在计算图中没有连接之外，还可能存在以下不太明显的情况会使`target`与`source`处于不连通的状态
1. 在“梯度磁带”中使用了非 TF 函数，例如 NumPy 函数
2. TF 默认整型变量是不可导的，若无意间声明了整型变量，会返回 None
3. 无意间将`Variable`替换成了`Tensor`对象，例如
    ```python
    x = tf.Variable(2.0)
    for epoch in range(2):
        with tf.GradientTape() as tape:
            y = x+1
        print(isinstance(x, tf.Variable))
        print(tape.gradient(y, x) is not None)
        x = x + 1
    """ ==>
    True
    True
    False
    False
    """
    ```

4. 对一个含有状态的对象求导
    `Tensor`对象一旦创建便不会再改变，至少在其值改变后，内存地址也随之改变，进而应视为另一个`Tensor`对象；`Tensor`对象含有值却不含有状态；目前为止讨论的所有操作都是无状态的，例如`tf.matmul`输出仅取决于其输入；

    然而`Variable`却拥有内部状态，即它的取值；当“梯度磁带”调用该变量时，首先会对其状态进行读取；然而“梯度磁带”只能读取当前状态，而不能读取导致当前状态的历史，即变量的当前状态阻碍了对其更早状态的梯度的计算，例如下面的例子：
    ```python
    x0 = tf.Variable(3.0)
    x1 = tf.Variable(0.0)
    with tf.GradientTape() as tape:
        x1.assign_add(x0)  # The tape starts recording from x1 but not x0
        y = x1**2

    # dy/dx0 should have been 2 * (x1 + x0), but it won't work.
    print(tape.gradient(y, x0))  # ==> None
    ```
    类似地，`tf.data.Dataset`迭代器和`tf.queues`也是含有状态的，进而会阻碍通过其自身的梯度流；

可以通过指明`gradient()`方法的`unconnected_gradients`参数，来控制在`target`和`source`不连通时的返回值——`tf.UnconnectedGradients.NONE`或`tf.UnconnectedGradients.ZERO`
## 5. 没有注册梯度的函数
一些`tf.Operations`会注册为不可微函数，进而会返回 None，其他一些并没有注册梯度；在[`tf.raw_ops`](https://www.tensorflow.org/api_docs/python/tf/raw_ops)处可以查看有哪些低层操作注册了梯度；

若通过没有注册梯度的操作来获取梯度，“梯度磁带”则会抛出异常而非返回 None；例如`tf.image.adjust_contrast`封装了`raw_ops.AdjustContrastv2`，该操作可以求导但并未对梯度进行实现；若想要通过这个操作进行微分，则需要人工实现梯度并利用`tf.RegisterGradient`注册梯度，或者使用其他操作重新实现该函数；
```python
image = tf.Variable([[[0.5, 0.0, 0.0]]])
delta = tf.Variable(0.1)
with tf.GradientTape() as tape:
    new_image = tf.image.adjust_contrast(image, delta)
try:
    print(tape.gradient(new_image, [image, delta]))
    assert False
except LookupError as e:
    print(f'A {type(e).__name__} is raised:\n{e}')
""" ==>
A LookupError is raised:
gradient registry has no entry for: AdjustContrastv2
"""
```
## 6. `GradientTape`对运行时性能的影响
在“梯度磁带”上下文管理器中执行操作会增加开销，这个开销通常很小，对大多即时执行而言并不会增加明显的运算成本；“梯度磁带”会使用内存存储中间结果，例如输入和输出等，以便反向传播时使用；为了提高效率，有些操作例如`ReLU`则不需要保留中间结果，向前传递时它们从计算图中去除，但如果指明了`persistent=True`，则在计算过程中“梯度磁带”不会丢弃任何内容，进而内存占用的峰值会更高；